In [1]:
import os

In [3]:
%pwd

'd:\\ML\\Projects\\Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\ML\\Projects\\Text-Summarizer'

**Entity -> means the return type of a function**

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """Data ingestion configuration class."""
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [56]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [57]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root]) # if error -> change self.config[artifacts_root]
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        print(self.config) # debug print
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
        )
        
        return data_ingestion_config

In [58]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [59]:
# defining required components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
        
    # to download data from url
    def download_file(self):
        # download file if doesn't exist
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with infor: \n {headers}")
        else:
            logger.info(f"File already downloaded, size: {get_size(Path(self.config.local_data_file))}")
            
            
            
    # zip file extraction
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file into the data directory
        function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
# creating pipeline (data download pipeline)
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-08 16:47:38,041: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-08 16:47:38,045: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-08 16:47:38,047: INFO: common: created new directory at: artifacts]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-12-08 16:47:38,050: INFO: common: created new directory at: artifacts/data_ingestion]
[2024-12-08 16:47:41,125: INFO: 1620229988: artifacts/data_ingestion/data.zip download! with infor: 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-S